In this notebook represented 4 indian boys from ages 18-23 talks, WER and the conclusion


In [2]:
import whisper
import glob
import jiwer
import pandas as pd
import IPython


In [8]:
def calc_wer(clean_text, hypotes, punct_marks=False, filler_words=True):
    # filler words used in language to spent time while thinking on something or just to make noise -_-
    # removing them might help to clearly find WER on to texts
    filler_words = ["oh", "um", "uh", "er", "ah"]
    clean_text = clean_text.strip().lower()
    hypotes = hypotes.strip().lower()    
    if punct_marks:
        for mark in (',', '-', ':', ';', '?', '!', "'"):
            clean_text = clean_text.replace(mark, '')
            hypotes = hypotes.replace(mark, '')
        clean_text = clean_text.replace('...', ' ')
        hypotes = hypotes.replace('...', ' ')
    if filler_words:
        for fil in filler_words:
            clean_text = clean_text.replace(fil, ' ')
            hypotes = hypotes.replace(fil, ' ')
    return jiwer.wer(clean_text, hypotes)

In [3]:
ind_4_accents = pd.read_csv('indian_4_accent_test_dt.csv')

In [32]:
res = []
model = whisper.load_model('tiny')
for name in ind_4_accents['name']:
    res.append(model.transcribe(f'4_indian_accents/{name}.mp3')['text'])

In [33]:
ind_4_accents['hypotes'] = res

In [34]:
ind_4_accents['WER_with_mark'] = [calc_wer(row['clean_text'], row['hypotes']) for i, row in ind_4_accents.iterrows()] 

ind_4_accents['WER_no_mark'] = [calc_wer(row['clean_text'], row['hypotes'], punct_marks=True) for i, row in ind_4_accents.iterrows()] 

Firstly I divided 2 types of WER with punctuation marks and without, because in English in a lot of cases, both variations of spellings are correct.

And as the results show there is difference, and after some manual checks as well whisper I came to conclussion that model not so good on understand placement of marks (not considering '...' symbols as per my understanding model shouldn't add this marks). Also this shows that as sentences from speech get complicated and long(meaning they will require more marks to represent correctly) model will perform badly.

In [35]:
ind_4_accents[["WER_with_mark", 'WER_no_mark']]

,WER_with_mark,WER_no_mark
0,0.301653,0.247934
1,0.266484,0.244505
2,0.495146,0.485437
3,0.379310,0.241379


On below example which 3rd in this small dataset speaker sometimes laughs while talking and results show that because of this whisper model got 'confused' and misunderstood a lot of words.
My thinking on this that as emotional gets speech more harder for this model to understand exact words. Also on this example a lot of words from hypotes text are out of context and random, my understanding on this that model only takes segment of audio and tries to find best word fiting that audio without cheking context.

In [26]:
IPython.display.Audio("4_indian_accents/Rahul.mp3")

In [4]:
res = []
model = whisper.load_model('medium')
for name in ind_4_accents['name']:
    res.append(model.transcribe(f'4_indian_accents/{name}.mp3')['text'])

In [6]:
ind_4_accents['hypotes'] = res

In [9]:
ind_4_accents['medium_WER_with_mark'] = [calc_wer(row['clean_text'], row['hypotes']) for i, row in ind_4_accents.iterrows()] 

ind_4_accents['medium_WER_no_mark'] = [calc_wer(row['clean_text'], row['hypotes'], punct_marks=True) for i, row in ind_4_accents.iterrows()] 

Same tests are done on 'medium model', results became better, but the symmetry of the results stayed the same

In [10]:
ind_4_accents[['medium_WER_with_mark', 'medium_WER_no_mark']]

,medium_WER_with_mark,medium_WER_no_mark
0,0.235537,0.169421
1,0.184066,0.162088
2,0.330097,0.330097
3,0.353448,0.215517
